In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,plot_confusion_matrix,multilabel_confusion_matrix

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import keras
import keras.backend as K
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
from keras import Model
from keras.layers.core import Lambda, Flatten, Dense
from keras.layers import Bidirectional, LSTM

Using TensorFlow backend.


In [3]:
df = pd.read_csv("DASH_2020_Drug_Data_train20_keyphrase.csv", usecols=['title','body','label_classification','body_keyphrase', 'title_keyphrase'])
group = df.groupby("label_classification")
print(group.count())

df['label_classification'][df['label_classification']=='Addicted'] = 1
df['label_classification'][df['label_classification']=='E-Recovery'] = 2
df['label_classification'][df['label_classification']=='M-Recovery'] = 3
df['label_classification'][df['label_classification']=='A-Recovery'] = 4
df['label_classification'][df['label_classification']=='Others'] = 5

# df=df.astype({'body': str, 'label_classification': 'int'})

df1 = df.loc[df['label_classification']==1][:5]
df2 = df.loc[df['label_classification']==2][:5]
df3 = df.loc[df['label_classification']==3][:5]
df4 = df.loc[df['label_classification']==4][:5]
df5 = df.loc[df['label_classification']==5][:5]

frames = [df1,df2,df3,df4,df5]
train = pd.concat(frames)
train= train.reset_index(drop=True).dropna()

test = pd.read_csv("DASH_2020_Drug_Data_test_keyphrase.csv", usecols=['title','body','label_classification','body_keyphrase','title_keyphrase'])
test['label_classification'][test['label_classification']=='Addicted'] = 1
test['label_classification'][test['label_classification']=='E-Recovery'] = 2
test['label_classification'][test['label_classification']=='M-Recovery'] = 3
test['label_classification'][test['label_classification']=='A-Recovery'] = 4
test['label_classification'][test['label_classification']=='Others'] = 5

print(len(train),len(test))

                      title  body  title_keyphrase  body_keyphrase
label_classification                                              
A-Recovery               20    20               20              20
Addicted                 20    20               20              20
E-Recovery               20    20               20              20
M-Recovery               20    20               20              20
Others                   20    20               20              20
25 1000


In [ ]:
import re
for i,row in train.iterrows():
    words1 = re.findall('[(](.*?)[,]', train.loc[i,'title_keyphrase'])
    words1 = [word[1:-1] for word in words1] if len(words1)<3 else [word[1:-1] for word in words1[:3]]
    cur_keys1 = '' if not words1 else ', '.join([word for word in words1])+'. '
    words2 = re.findall('[(](.*?)[,]', train.loc[i,'body_keyphrase'])
    words2 = [word[1:-1] for word in words2] if len(words2)<3 else [word[1:-1] for word in words2[:3]]
    cur_keys2 = '' if not words2 else ', '.join([word for word in words2])+'. '
    train.loc[i,'body'] = train.loc[i,'title']+'. '+cur_keys1+cur_keys2+train.loc[i,'body'] if len(train.loc[i,'title'])>0 else cur_keys1+cur_keys2+train.loc[i,'body']

for i,row in test.iterrows():
    words1 = re.findall('[(](.*?)[,]', test.loc[i,'title_keyphrase'])
    words1 = [word[1:-1] for word in words1] if len(words1)<3 else [word[1:-1] for word in words1[:3]]
    cur_keys1 = '' if not words1 else ', '.join([word for word in words1])+'. '
    words2 = re.findall('[(](.*?)[,]', test.loc[i,'body_keyphrase'])
    words2 = [word[1:-1] for word in words2] if len(words2)<3 else [word[1:-1] for word in words2[:3]]
    cur_keys2 = '' if not words2 else ', '.join([word for word in words2])+'. '
    test.loc[i,'body'] = test.loc[i,'title']+'. '+cur_keys1+cur_keys2+test.loc[i,'body'] if len(test.loc[i,'title'])>0 else cur_keys1+cur_keys2+test.loc[i,'body']

train

In [5]:
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-large/4'
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.load(module_url)

In [6]:
input_text1 = Input(shape=(512,))
x = Dense(256, activation='relu')(input_text1)
x = Dropout(0.4)(x)
x = BatchNormalization()(x)
x = Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = Dropout(0.4)(x)
dense_layer = Dense(128, name='dense_layer')(x)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)

model=Model(inputs=[input_text1], outputs=norm_layer)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_layer (Dense)          (None, 128)               8320  

In [7]:
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Layer

# Input for anchor, positive and negative images
in_a = Input(shape=(512,))
in_p = Input(shape=(512,))
in_n = Input(shape=(512,))

# Output for anchor, positive and negative embedding vectors
# The nn4_small model instance is shared (Siamese network)
emb_a = model(in_a)
emb_p = model(in_p)
emb_n = model(in_n)

class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        a, p, n = inputs
        p_dist = K.sum(K.square(a-p), axis=-1)
        n_dist = K.sum(K.square(a-n), axis=-1)
        return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss

# Layer that computes the triplet loss from anchor, positive and negative embedding vectors
triplet_loss_layer = TripletLossLayer(alpha=0.4, name='triplet_loss_layer')([emb_a, emb_p, emb_n])

# Model that can be trained with anchor, positive negative images
nn4_small2_train = Model([in_a, in_p, in_n], triplet_loss_layer)

In [8]:
from tqdm import tqdm
# creating the necessary datastructures for selcting triplets
unique_train_label=np.array(train['label_classification'].unique().tolist())
labels_train=np.array(train['label_classification'].tolist())
map_train_label_indices = {label: np.flatnonzero(labels_train == label) for label in unique_train_label}

def get_triplets(unique_train_label,map_train_label_indices):
    label_l, label_r = np.random.choice(unique_train_label, 2, replace=False)
    a, p = np.random.choice(map_train_label_indices[label_l],2, replace=False)
    n = np.random.choice(map_train_label_indices[label_r])
    return a, p, n

def get_triplets_batch(k,train_set,unique_train_label,map_train_label_indices,embed):

    while True:
        idxs_a, idxs_p, idxs_n = [], [], []
        for _ in range(k):
            a, p, n = get_triplets(unique_train_label,map_train_label_indices)
            idxs_a.append(a)
            idxs_p.append(p)
            idxs_n.append(n)

        a=train_set.iloc[idxs_a].values.tolist()
        b=train_set.iloc[idxs_p].values.tolist()
        c=train_set.iloc[idxs_n].values.tolist()

        a = embed(a)
        p = embed(b)
        n = embed(c)
        # return train_set[idxs_a], train_set[idxs_p], train_set[idxs_n]
        yield [a,p,n], []

In [ ]:
nn4_small2_train.compile(loss=None, optimizer='adam')
history = nn4_small2_train.fit(get_triplets_batch(32,train['body'],unique_train_label,map_train_label_indices,embed), epochs=50,steps_per_epoch=10)
nn4_small2_train.save('drug_few-shot5_key(both)+title+text')

In [10]:
from tqdm import tqdm
# test=test.sample(n=300,random_state=200).reset_index(drop=True)
#getting the embeddings from the model
encoded = []
for val in tqdm(np.array(train['body'].values.tolist())):
    encoded.append(embed([val])['outputs'][0])
X_train_input = {'input_1':tf.convert_to_tensor(encoded)}

encoded = []
for val in tqdm(np.array(test['body'].values.tolist())):
    encoded.append(embed([val])['outputs'][0])
X_test_input = {'input_1':tf.convert_to_tensor(encoded)}

X_train = model.predict(X_train_input)
X_test = model.predict(X_test_input)

y_train = np.array(train['label_classification'].values.tolist())
y_test = np.array(test['label_classification'].values.tolist())

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
svc = LinearSVC()

knn.fit(X_train, y_train)
svc.fit(X_train, y_train)

y_pred_knn = knn.predict(X_test)
acc_knn = accuracy_score(y_test, y_pred_knn)
y_pred_svc = svc.predict(X_test)
acc_svc = accuracy_score(y_test, y_pred_svc)

print(f'KNN accuracy = {acc_knn}, SVM accuracy = {acc_svc}')

100%|██████████| 1000/1000 [00:20<00:00, 48.94it/s]


KNN accuracy = 0.517, SVM accuracy = 0.519


In [11]:
from sklearn import metrics
print(metrics.classification_report(list(y_test), list(y_pred_knn), digits=4))
with open('drug_result_few-shot5_key(both)+title+text.txt','w') as f:
    f.write('KNN accuracy: '+str(acc_knn)+'\n'+'SVM accuracy: '+str(acc_svc))

              precision    recall  f1-score   support

           1     0.5104    0.7350    0.6025       200
           2     0.5000    0.3950    0.4413       200
           3     0.3807    0.4150    0.3971       200
           4     0.5357    0.5250    0.5303       200
           5     0.7357    0.5150    0.6059       200

    accuracy                         0.5170      1000
   macro avg     0.5325    0.5170    0.5154      1000
weighted avg     0.5325    0.5170    0.5154      1000

